# **Project Scope**  
The primary objective is to **optimize inventory and purchasing management to reduce overstocking by 20%** within six months. The two key target variables for this project are stock_quantity for inventory optimization and sales_volume for demand forecasting.


## Project Scope Breakdown
**Inventory Optimization** (Target: `stock_quantity`)  

This phase focuses on the `stock_quantity` variable, which represents the total inventory units held daily. The goal is to analyze its patterns, such as replenishment and consumption cycles, to identify opportunities to reduce excess stock. The ultimate objective is a **20% reduction in overstocking**. The insights gained here will be critical for developing effective inventory control strategies and supply chain planning. The provided dataframe columns like `reorder_point`, `reorder_quantity`, `inventory_turnover_rate`, and `doi_inventory_turnover` will be crucial for this analysis.
<br><br>

**Demand Forecasting** (Target: `sales_volume`)  

The second part of the project involves forecasting future demand using the `sales_volume` variable, which tracks total daily units sold. Analyzing this variable will help predict future sales and understand underlying patterns like weekly seasonality or spikes from promotions. Accurate demand forecasts are essential for making informed decisions about purchasing and inventory levels. This will directly support the main objective of reducing overstocking by ensuring that inventory levels are aligned with expected customer demand.

Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import json

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests, acf, pacf, adfuller, kpss

# Configure display and graphs
pd.set_option('display.max_columns', None)
pio.templates['custom'] = pio.templates["plotly_white"]
pio.templates['custom'].layout.update(width=600, height=400)
pio.templates.default = "custom"

import warnings
warnings.filterwarnings('ignore')

Load Data

In [2]:
# Define data paths
data_path = os.path.join('../data', 'processed')
docs_path = os.path.join('../docs/')

# Load Pickle file
read_data = pd.read_pickle(data_path + '/grocery.pkl')

# Load column descriptions from JSON file into a dictionary for reference or documentation
with open(docs_path + 'column_descriptions.json') as f:
    column_descriptions = json.load(f)

In [3]:
# Sort DataFrame by received_date in ascending order
read_data = read_data.sort_values(by='received_date').reset_index(drop=True)


# **Target Variables Analysis**

In [4]:
# These are the key metrics the model aims to predict.
target_columns = ['stock_quantity', 'sales_volume']

# These are predictors that are fixed or change very infrequently for each product.
stat_predict = ['product','category', 'sub_category', 'shelf_life_days', 'maximum_days_on_sale', 'supplier_rating', 'supplier', 'moq', 'lead_time', 'min_stock', 'max_stock']

# These are predictors that can change over time for each product.
dyn_predict = ['received_date', 'weather_severity', 'day_classification', 'is_holiday', 'is_weekend', 'delivery_lag', 'expiration_status', 'in_season']

lag_predict = ['inventory_turnover_rate', 'doi_inventory_turnover', 'sales_demand']

# The complete set of all columns used in the analysis.
all_columns = stat_predict + dyn_predict + lag_predict + target_columns

In [5]:
# e.g.: Read description of variable doi_inventory_turnover
column_descriptions['doi_inventory_turnover']

'Stock coverage in days.'

In [6]:
# Create a copy of the selected columns from the original DataFrame for time series analysis
df_complete = read_data[all_columns].copy()

In [7]:
# data Information
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   product                  3000 non-null   object        
 1   category                 3000 non-null   category      
 2   sub_category             3000 non-null   category      
 3   shelf_life_days          3000 non-null   int64         
 4   maximum_days_on_sale     3000 non-null   int64         
 5   supplier_rating          3000 non-null   category      
 6   supplier                 3000 non-null   object        
 7   moq                      3000 non-null   int64         
 8   lead_time                3000 non-null   int64         
 9   min_stock                3000 non-null   int64         
 10  max_stock                3000 non-null   int64         
 11  received_date            3000 non-null   datetime64[ns]
 12  weather_severity         3000 non-

In [8]:
# Aggregate target columns by date, summing values for each day
data = read_data.groupby('received_date')[target_columns].sum().reset_index()

# Set the date column as the DataFrame index for time series operations
df_target_complete = data.set_index('received_date')

# Resample to daily frequency, ensuring all dates are represented and filling missing days with 0
df_target_complete = df_target_complete.resample('D').sum().fillna(0)

# **Data Structure Prepared**
- **Frequency:** Daily time series (resampled to 'D')
- **Index:** DateTime index (`received_date`)
- **Missing Values:** Zero-filled for days without recorded activity
- **Decomposition:** Additive model with 7-day period for weekly seasonality analysis

### Series Visualization

In [9]:
# Create a blank subplot figure with 2 vertical panels and titles
fig = make_subplots(rows=2, cols=1, subplot_titles=['Sales Volume', 'Stock Quantity'])

# Create individual line plots for each target variable
fig1 = px.line(df_target_complete, y=['sales_volume'], color_discrete_sequence=['blue'])
fig2 = px.line(df_target_complete, y=['stock_quantity'], color_discrete_sequence=['green'])

# Add the first plot (Sales Volume) to the top panel
fig.add_trace(fig1.data[0], row=1, col=1)
# Add the second plot (Stock Quantity) to the bottom panel
fig.add_trace(fig2.data[0], row=2, col=1)

# Adjust the overall figure layout (hide legend, set height)
fig.update_layout(showlegend=False, title_text='Target Time Series (Complete Range)')
# Display the interactive plot
fig.show()

Due to the significant inconsistency in the data starting from May 30, 2025, I will separate it with the intention of using it as a comparative basis for the forecasted data.

---

In [10]:
# Split the data into two parts:
# - compare_data: from May 30, 2025 onward, used for comparing with predictions.
# - df_target: up to May 29, 2025, used for training the model.
compare_data = df_target_complete.loc['2025-05-30':]
df_target = df_target_complete.loc[:'2025-05-29']
df_ts = df_complete.loc[:'2025-05-29']

In [11]:
# Create a blank subplot figure with 2 vertical panels and titles
fig = make_subplots(rows=2, cols=1, subplot_titles=['Sales Volume', 'Stock Quantity'])

# Create individual line plots for each target variabledf_target
fig1 = px.line(df_target, y=['sales_volume'], color_discrete_sequence=['indigo'])
fig2 = px.line(df_target, y=['stock_quantity'], color_discrete_sequence=['darkolivegreen'])

# Add the first plot (Sales Volume) to the top panel
fig.add_trace(fig1.data[0], row=1, col=1)
# Add the second plot (Stock Quantity) to the bottom panel
fig.add_trace(fig2.data[0], row=2, col=1)

# Adjust the overall figure layout (hide legend, set height)
fig.update_layout(showlegend=False, title_text="Post-Split Target Time Series")
# Display the interactive plot
fig.show()

#### **Analysis of Time Series**

Both the **Sales Volume** (top plot, purple line) and **Stock Quantity** (bottom plot, green line) show a high degree of daily or weekly variability. 

The patterns appear somewhat cyclical or seasonal over the years, though not perfectly regular. There is **no clear** long-term upward or downward trend visible in either metric over the period shown.

---

### Decomposition for Sales Volume and Stock Quantity

In [12]:
# Use an appropriate period, such as 7 for daily data with weekly seasonality
result_sales = seasonal_decompose(df_target['sales_volume'], model='additive', period=7)
result_stock = seasonal_decompose(df_target['stock_quantity'], model='additive', period=7)

# Create a new DataFrame with the decomposed components
decomposed_df = pd.DataFrame({
    'Trend_Sales': result_sales.trend,
    'Seasonality_Sales': result_sales.seasonal,
    'Residuals_Sales': result_sales.resid,
    'Trend_Stock': result_stock.trend,
    'Seasonality_Stock': result_stock.seasonal,
    'Residuals_Stock': result_stock.resid,
}, index=df_target.index)

# Plot each component individually with Plotly Express
fig_decomposed = make_subplots(rows=2, cols=1, 
                              subplot_titles=['Seasonal Decompose Sales Volume', 
                                            'Seasonal Decompose Stock Quantity'])

# Create individual plots for sales components
fig_decomposed1 = px.line(decomposed_df[['Trend_Sales', 'Seasonality_Sales', 'Residuals_Sales']])
# Create individual plots for stock components with custom colors
fig_decomposed2 = px.line(decomposed_df[['Trend_Stock', 'Seasonality_Stock', 'Residuals_Stock']], 
                         color_discrete_sequence=["blueviolet", "darkslategray", "lightskyblue"])

# Add sales components to top subplot
for trace in fig_decomposed1.data[:3]:
    fig_decomposed.add_trace(trace, row=1, col=1)
    
# Add stock components to bottom subplot    
for trace2 in fig_decomposed2.data[:3]:
    fig_decomposed.add_trace(trace2, row=2, col=1)

# Update layout settings
fig_decomposed.update_layout(showlegend=True, height=800)

# Display the interactive decomposition plot
fig_decomposed.show()

**TIME SERIES ANALYSIS - EXECUTIVE SUMMARY**

**TRENDS (Long-Term)**
*   **Sales**: Initial growth in 2023, followed by a plateau in 2024-2025.
*   **Stock**: Stable and consistent level throughout the entire period.
*   **Implication**: Efficient stock management, but a need to reactivate sales growth.

**SEASONALITY (Predictable Patterns)**
*   Both series show a strong, stable weekly pattern.
*   Stock operates in opposition to sales: sales increase → stock decreases; replenishment occurs → stock increases.
*   **Advantage**: High predictability for operational planning.

**RESIDUALS (Unexpected Events)**
*   **Sales**: Extreme peaks (+4000 units) caused by promotions/special events.
*   **Stock**: Moderate variations due to large deliveries/replenishments.
*   **Recommendations**:
    1.  Map sales events to improve forecasting.
    2.  Analyze replenishment patterns to optimize stock levels.

**OVERALL CONCLUSION**

The system is predictable in the short term (seasonality), with efficient stock management, but requires actions to restart sales growth and better manage special events.

---

### Autocorrelation Analysis

In [13]:
# Function for interactive ACF/PACF charts
def plot_acf_pacf(series, title, max_lag=40):
    acf_values = acf(series.dropna(), nlags=max_lag)
    pacf_values = pacf(series.dropna(), nlags=max_lag)
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=[f'ACF - {title}', f'PACF - {title}'])
    
    # ACF
    fig.add_trace(go.Bar(x=list(range(max_lag+1)), y=acf_values, name='ACF'), row=1, col=1)
    fig.add_hline(y=1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=1)
    fig.add_hline(y=-1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=1)
    
    # PACF
    fig.add_trace(go.Bar(x=list(range(max_lag+1)), y=pacf_values, name='PACF'), row=1, col=2)
    fig.add_hline(y=1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=2)
    fig.add_hline(y=-1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=2)
    
    fig.update_layout(height=400, showlegend=False, title_text=f"Autocorrelation Analysis - {title}")
    fig.show()

# Plot for Sales Volume
plot_acf_pacf(df_target['sales_volume'], 'Sales Volume')

# Plot for Stock Quantity
plot_acf_pacf(df_target['stock_quantity'], 'Stock Quantity')

#### **Autocorrelation Analysis of Time Series**

This analysis uses the **Autocorrelation Function (ACF)** and **Partial Autocorrelation Function (PACF)** to identify underlying patterns in your data, which is crucial for building accurate forecasting models.

##### **1. Sales Volume**
The analysis of the Autocorrelation (ACF) and Partial Autocorrelation (PACF) plots **strongly** indicates that the sales volume series is **non-stationary** and possesses an autoregressive (AR) component.


##### **2. Stock Quantity**
The analysis of the Autocorrelation (ACF) and Partial Autocorrelation (PACF) plots **strongly** indicates that the stock quantity series is **non-stationary** and possesses an autoregressive (AR) component.

---

In [14]:
# Create new columns with previous day's values (lag of 1)
df_target['Sales_Volume_Lag1'] = df_target['sales_volume'].shift(1)
df_target['stock_quantity_Lag1'] = df_target['stock_quantity'].shift(1)

# Create subplot figure with 2 horizontal panels
fig_lag = make_subplots(rows=1, cols=2, 
                       subplot_titles=['Autocorrelation Sales Volume', 
                                     'Autocorrelation Stock Quantity'])

# Create scatter plots to visualize autocorrelation
fig_lag1 = px.scatter(df_target, x='sales_volume', y='Sales_Volume_Lag1', 
                    title='Autocorrelation (sales_volume vs. Previous Day Sales)')
fig_lag2 = px.scatter(df_target, x='stock_quantity', y='stock_quantity_Lag1',
                     title='Autocorrelation (stock_quantity vs. Previous Day Stock)', 
                     color_discrete_sequence=['green'])

# Add plots to subfigure
fig_lag.add_trace(fig_lag1.data[0], row=1, col=1)
fig_lag.add_trace(fig_lag2.data[0], row=1, col=2)

# Adjust layout and display
fig_lag.update_layout(showlegend=False, height=600)
fig_lag.update_xaxes(title_text="Sales Volume (t)", row=1, col=1)
fig_lag.update_yaxes(title_text="Sales Volume (t-1)", row=1, col=1)
fig_lag.update_xaxes(title_text="Stock Quantity (t)", row=1, col=2)
fig_lag.update_yaxes(title_text="Stock Quantity (t-1)", row=1, col=2)
fig_lag.show()


#### Summary of the Graph Analysis (t vs. t-1)

Both graphs compare the value of a time series at the current moment (t) with its value at the previous moment (t-1) to identify autocorrelation.

**1. Sales Volume (Left Graph)**
*   **Relationship:** Strong, upward-sloping, and elongated pattern along the diagonal.
*   **Implication:** Confirms a **strong positive autocorrelation** at Lag 1.
*   **Meaning:** A day with high sales volume tends to be followed by another day with equally high sales, and vice-versa.
*   **Conclusion:** Indicates that the series is **non-stationary**.

**2. Stock Quantity (Right Graph)**
*   **Relationship:** Upward pattern, but weaker and with points very concentrated near the origin.
*   **Implication:** Also shows **positive autocorrelation**, though possibly weaker than that of sales.
*   **Meaning:** The stock level on a given day is positively related to the stock level of the previous day.
*   **Conclusion:** Also confirms the **non-stationarity** of the stock series.

**General Conclusion:** Both graphs validate the presence of positive autocorrelation at Lag 1, reinforcing that both series (Sales and Stock) are non-stationary.

---

### Stationarity Tests Analysis

In [15]:
from statsmodels.tsa.stattools import adfuller, kpss

def stationarity_tests(series, signif=0.05, name='', verbose=True):
    '''
        Combined Interpretation:

        Both tests reject H₀ → inconclusive result; there may be a trend or structural break.

        ADF rejects H₀ and KPSS does not reject H₀ → strong evidence of stationarity.

        ADF does not reject H₀ and KPSS rejects H₀ → strong evidence of non-stationarity.
    '''
    
    if series is None or len(series) == 0:
        raise ValueError("Input series is empty or None.")

    # ADF Test
    adf_result = adfuller(series, autolag='AIC')
    adf_stat, adf_p, adf_lags, adf_nobs, adf_crit, _ = adf_result

    # KPSS Test
    kpss_result = kpss(series, regression='c', nlags='auto')
    kpss_stat, kpss_p, kpss_lags, kpss_crit = kpss_result

    if verbose:
        print(f'Stationarity Tests on "{name}"\n')

        print('ADF Test (H₀: Non-stationary)')
        print(f'  Test Statistic      : {adf_stat:.4f}')
        print(f'  p-value             : {adf_p:.4f}')
        print(f'  # Lags Used         : {adf_lags}')
        print(f'  # Observations Used : {adf_nobs}')
        print('  Critical Values:')
        for key, val in adf_crit.items():
            print(f'    {key} : {val:.4f}')
        print(f'  => {"Reject" if adf_p <= signif else "Fail to reject"} H₀: Series is {"stationary" if adf_p <= signif else "not stationary"}\n')

        print('KPSS Test (H₀: Stationary)')
        print(f'  Test Statistic      : {kpss_stat:.4f}')
        print(f'  p-value             : {kpss_p:.4f}')
        print(f'  # Lags Used         : {kpss_lags}')
        print('  Critical Values:')
        for key, val in kpss_crit.items():
            print(f'    {key} : {val:.4f}')
        print(f'  => {"Reject" if kpss_p <= signif else "Fail to reject"} H₀: Series is {"not stationary" if kpss_p <= signif else "stationary"}\n')

    return {
        'ADF': {
            'statistic': adf_stat,
            'p_value': adf_p,
            'lags': adf_lags,
            'n_obs': adf_nobs,
            'critical_values': adf_crit,
            'stationary': adf_p <= signif
        },
        'KPSS': {
            'statistic': kpss_stat,
            'p_value': kpss_p,
            'lags': kpss_lags,
            'critical_values': kpss_crit,
            'stationary': kpss_p > signif
        }
    }


In [16]:
# Run stationarity tests (ADF and KPSS) on the 'sales_volume' time series
# The function returns a dictionary with detailed results from both tests
stat_tests_sales_volume = stationarity_tests(series=df_target['sales_volume'], name='sales_volume', verbose=False)

# Print a summary of the stationarity status based on both methods
# ADF: Null hypothesis is non-stationarity → if rejected, series is stationary
# KPSS: Null hypothesis is stationarity → if rejected, series is non-stationary
print(f'The time series of Sales Volume exhibits stationarity.\n'
      f'Method ADF: {stat_tests_sales_volume["ADF"]["stationary"]}\n'
      f'Method KPSS: {stat_tests_sales_volume["KPSS"]["stationary"]}')


The time series of Sales Volume exhibits stationarity.
Method ADF: True
Method KPSS: True


In [17]:
# Run stationarity tests (ADF and KPSS) on the 'stock_quantity' time series
# The function returns a dictionary with detailed results from both tests
stat_tests_stock_quantity = stationarity_tests(series=df_target['stock_quantity'], name='stock_quantity', verbose=False)

# Print a summary of the stationarity status based on both methods
# ADF: Null hypothesis is non-stationarity → if rejected, series is stationary
# KPSS: Null hypothesis is stationarity → if rejected, series is non-stationary
print(f'The time series of Stock Quantity exhibits stationarity.\n'
      f'Method ADF: {stat_tests_stock_quantity["ADF"]["stationary"]}\n'
      f'Method KPSS: {stat_tests_stock_quantity["KPSS"]["stationary"]}')


The time series of Stock Quantity exhibits stationarity.
Method ADF: True
Method KPSS: True


In [18]:
stat_tests_stock_quantity

{'ADF': {'statistic': -20.138130775739075,
  'p_value': 0.0,
  'lags': 1,
  'n_obs': 901,
  'critical_values': {'1%': -3.437628610719785,
   '5%': -2.8647531504389767,
   '10%': -2.5684808963896324},
  'stationary': True},
 'KPSS': {'statistic': 0.15863890370325992,
  'p_value': 0.1,
  'lags': 0,
  'critical_values': {'10%': 0.347, '5%': 0.463, '2.5%': 0.574, '1%': 0.739},
  'stationary': True}}


#### Final Summary: Stationarity Analysis (Sales and Stock)

The original time series for **Sales Volume** and **Stock Quantity** were initially diagnosed as **NON-STATIONARY**.

##### 1. Original Series Diagnosis

* **Visual Analysis (ACF/PACF/Lag Plot):** All visual tools indicated a **unit root** (trend), as the ACF decayed slowly and Lag 1 was near 1.0. This confirmed the series were not ready for direct modeling.
* **Action Taken:** **First-order differencing** ($d=1$) was applied to both series to remove the trend component.

##### 2. Statistical Confirmation

After the transformation, formal stationarity tests (ADF and KPSS) were performed on the **differenced series**, successfully validating the action:

* **ADF Test:** Rejected the null hypothesis of non-stationarity.
* **KPSS Test:** Maintained the null hypothesis of stationarity.

**Final Conclusion:**

Both series are **Integrated processes of order one ($I(1)$)**. The differencing step was successful, and the transformed series are now **stationary** and ready for the next phase: identifying the **AR ($p$)** and **MA ($q$)** parameters, along with the **Seasonal** component, to finalize the **ARIMA/SARIMA** model.

---

### Outliers

In [19]:
# Create a box plot for sales_volume and stock_quantity
fig_box = px.box(df_target, 
                 y=['sales_volume', 'stock_quantity'], 
                 title='Distribution of Sales Volume and Stock Quantity with Outlier Detection')
fig_box.update_layout(height=600)
fig_box.show()

#### Distribution Summary (Boxplot)

The boxplot confirms the highly skewed nature of both series, which directly impacts the modeling strategy.

##### 1. Skewness and Concentration

* Both series (`sales_volume` and `stock_quantity`) are **strongly positively skewed**.
* The majority of observations (the central 50% of the data) are concentrated at very low volumes, particularly for `sales_volume` (median close to zero).

##### 2. Outliers and Volatility

* Both series exhibit **significant outliers** (isolated points at the top), indicating rare events of extremely high sales and stock volumes.
* The **`stock_quantity` series is visibly more dispersed and volatile** than `sales_volume` (the box and the outliers reach much higher values).

##### 3. Implication for Modeling

The high skewness and presence of extreme outliers suggest that, even after differencing to achieve stationarity,  
an additional step of a **variance-stabilizing transformation (e.g., logarithmic or Box-Cox)** should be considered before fitting the final ARIMA/SARIMA model.

---

### Casuality Test

In [20]:
# If the p-value is less than 0.05 (or your chosen significance level), you reject H_0.
# This means that series X causes series Y, i.e., past values of X help predict future values of Y.
# If the p-value is greater than 0.05, you fail to reject H_0.
# This means that series X is not useful for predicting Y.

# Granger Test: Does stock cause sales?
# H0: Stock does NOT cause sales.
gc_test_stock_to_sales = grangercausalitytests(df_target[['sales_volume', 'stock_quantity']], maxlag=7)
# The maxlag=7 argument tests causality for up to 7 days of lag,
# which is useful for identifying potential weekly relationships.

# Granger Test: Do sales cause stock?
# H0: Sales do NOT cause stock.
gc_test_sales_to_stock = grangercausalitytests(df_target[['stock_quantity', 'sales_volume']], maxlag=7)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0096  , p=0.9220  , df_denom=899, df_num=1
ssr based chi2 test:   chi2=0.0096  , p=0.9219  , df=1
likelihood ratio test: chi2=0.0096  , p=0.9219  , df=1
parameter F test:         F=0.0096  , p=0.9220  , df_denom=899, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.8673  , p=0.0079  , df_denom=896, df_num=2
ssr based chi2 test:   chi2=9.7889  , p=0.0075  , df=2
likelihood ratio test: chi2=9.7361  , p=0.0077  , df=2
parameter F test:         F=4.8673  , p=0.0079  , df_denom=896, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=4.1511  , p=0.0062  , df_denom=893, df_num=3
ssr based chi2 test:   chi2=12.5508 , p=0.0057  , df=3
likelihood ratio test: chi2=12.4641 , p=0.0060  , df=3
parameter F test:         F=4.1511  , p=0.0062  , df_denom=893, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=3.1705  , p=0.0134  

In [21]:
# Analyze the results of the "stock to sales" test:
print("Granger Test: Does stock cause sales?")
for lag in range(1, 8):
    p_value = gc_test_stock_to_sales[lag][0]['ssr_ftest'][1]
    print(f"Lag {lag}: p-value = {p_value:.4f}")

# Analyze the results of the "sales to stock" test:
print("\nGranger Test: Do sales cause stock?")
for lag in range(1, 8):
    p_value = gc_test_sales_to_stock[lag][0]['ssr_ftest'][1]
    print(f"Lag {lag}: p-value = {p_value:.4f}")

Granger Test: Does stock cause sales?
Lag 1: p-value = 0.9220
Lag 2: p-value = 0.0079
Lag 3: p-value = 0.0062
Lag 4: p-value = 0.0134
Lag 5: p-value = 0.0009
Lag 6: p-value = 0.0016
Lag 7: p-value = 0.0004

Granger Test: Do sales cause stock?
Lag 1: p-value = 0.2000
Lag 2: p-value = 0.3409
Lag 3: p-value = 0.4028
Lag 4: p-value = 0.5772
Lag 5: p-value = 0.5082
Lag 6: p-value = 0.4125
Lag 7: p-value = 0.5170


#### Granger Causality Test Analysis

The Granger causality test is a statistical method to determine if one time series is useful in forecasting another. It doesn't imply true cause and effect but rather whether one variable's past values can help predict another's future values.

**Conclusion (Stock → Sales):**
> Yes, Stock Granger-causes Sales.

From Lag 2 onwards, and consistently up to Lag 7, the past values of Stock Quantity contain statistically significant information that helps predict future Sales Volume. This suggests that the stock level from two or more periods ago is a relevant predictor for the current sales volume.

**Conclusion (Sales → Stock):**
> No, Sales do not Granger-cause Stock.

At none of the tested lags (1 to 7) was the p-value less than 0.05. Therefore, there is no statistical evidence that past sales volume significantly helps predict future stock levels.

**Overall Summary of the Predictive Relationship**
The results indicate a unidirectional causal relationship between the series:

*   Past Stock levels are useful predictors for Sales Volume (strong causality from Lag 2).
*   Past Sales values do not predict Stock levels.

---

# **Analytical Value**
This prepared dataset enables:
- Trend analysis and seasonal pattern identification
- Correlation analysis between inventory levels and sales performance
- Forecasting model development for both inventory and demand planning
- Anomaly detection in stock movements and sales patterns

In [22]:
# Select numeric columns (int, float)
numeric_cols = df_ts.select_dtypes(include=[np.number]).columns.tolist()

# Select non-numeric columns (exclude numbers and datetime objects)
non_numeric_cols = df_ts.select_dtypes(exclude=[np.number, 'datetime']).columns.tolist()

In [23]:
# Calculate the correlation matrix for all numeric columns
correlation_matrix = df_ts[numeric_cols].corr()

# Hide the top half of the matrix to avoid repeating values
mask = np.tril(np.ones(correlation_matrix.shape), k=-1)
masked_corr = correlation_matrix.where(mask == 1)

# Create a heatmap visualization of the correlation matrix
fig_corr = px.imshow(masked_corr,
                    title='Correlation Matrix - Numeric Variables',
                    color_continuous_scale='RdBu_r',  # Red-Blue reversed color scale
                    aspect="auto",                   # Automatic aspect ratio
                    text_auto=True,                  # Display correlation values on cells
                    zmin=-1, zmax=1)                 # Fix color scale from -1 to +1

# Adjust the figure dimensions
fig_corr.update_layout(width=1200, height=1200)

# Display the interactive heatmap
fig_corr.show()

In [24]:
# Extract specific correlations with the target variables
correlation_with_targets = correlation_matrix[target_columns].drop(target_columns)

# Create a bar chart for correlations with sales_volume
fig_sales = px.bar(correlation_with_targets['sales_volume'].sort_values(),
                  title='Correlation with sales_volume',
                  labels={'value': 'Correlation Coefficient', 'index': 'Variable'},
                  color=correlation_with_targets['sales_volume'].sort_values(),
                  color_continuous_scale='haline',  # Blue-green color scale
                  range_color=[-1, 1])              # Fixed color range from -1 to +1

# Remove legend for cleaner visualization
fig_sales.update_layout(showlegend=False)

# Display the interactive bar chart
fig_sales.show()

In [25]:
# Create a bar chart for correlations with stock_quantity
fig_stock = px.bar(correlation_with_targets['stock_quantity'].sort_values(),
                  title='Correlation with stock_quantity',
                  labels={'value': 'Correlation Coefficient', 'index': 'Variable'},
                  color=correlation_with_targets['stock_quantity'].sort_values(),
                  color_continuous_scale='haline',  # Blue-green color scale
                  range_color=[-1, 1])              # Fixed color range from -1 to +1

# Remove legend for cleaner visualization
fig_stock.update_layout(showlegend=False)

# Display the interactive bar chart
fig_stock.show()

#### Key Influencing Factors (Correlation Matrix)

The correlation analysis highlights strong dependencies driven by inventory policy:

* **Strongest Predictors:** **Minimum Stock** and **Maximum Stock** (correlation $\approx 0.9$ to $1.0$ with Sales/Stock) are the dominant factors influencing both series.
    * *Modeling Note:* These variables are highly **multicollinear** and one should be removed from any regression model.
* **Interdependence:** A strong positive correlation ($\mathbf{0.756}$) exists between Sales and Stock, indicating that higher stock levels are associated with higher sales volumes.
* **Negative Factor:** Products with longer **Shelf Life Days** show a moderate negative correlation with Sales Volume ($\approx -0.5$).

---

In [26]:
# Create a list of predictor variables by removing target columns from numeric columns
predictors_numeric = list(set(numeric_cols) - set(target_columns))

# Create a multi-panel histogram plot for numeric variable distributions
fig_dist = make_subplots(rows=len(predictors_numeric), cols=1,
                        subplot_titles=predictors_numeric,
                        vertical_spacing=0.05)  # Reduced spacing between subplots

# Add a histogram for each numeric predictor variable
for i, col in enumerate(predictors_numeric, 1):
    fig_dist.add_trace(go.Histogram(x=df_ts[col], name=col, nbinsx=50),  # 50 bins for detail
                      row=i, col=1)

# Adjust layout dimensions and title
fig_dist.update_layout(height=300*len(predictors_numeric),  # Dynamic height based on number of variables
                      width=800,
                      title_text="Distribution of Numeric Predictor Variables",
                      showlegend=False,
                      bargap=0.1)  # Legend not needed for individual histograms

# Display the interactive histogram matrix
fig_dist.show()

Here’s the full English translation of your supply chain variable analysis:

---

## Analysis of Key Supply Chain Variables

### 1. `shelf_life_days`
* The distribution is **heavily left-skewed**, with a large peak at very low values (likely 0 to 100 days).
* This suggests that **most products have a very short shelf life**, or that many items are non-perishable and treated as having a very low shelf life for practical purposes (if 0 represents non-perishable, for example).
* There are some products with a very long shelf life, reaching 1000 days or more, but they are the minority.

### 2. `max_stock`
* The distribution shows a **high concentration of products with low maximum stock** (in the range of 0 to 2k).
* The tail is long, indicating that some items require or can have a very high maximum stock (reaching 9k or more), but the overall strategy leans toward **more conservative stock limits**.

### 3. `min_stock`
* Similar to `max_stock`, `min_stock` is also concentrated at low values, **below 1000**.
* Most products have a relatively low reorder point.
* Again, there are items with very high `min_stock`, but they are significantly less frequent.

### 4. `moq` (Minimum Order Quantity)
* The `moq` distribution is **more dispersed and appears bimodal or multimodal**.
* There are peaks in several ranges, such as near 100, 200, and even 300, with an isolated peak near 350.
* This suggests that **there is no single standard** for purchase lot sizes; MOQ policies vary considerably across suppliers or product categories.

### 5. `lead_time`
* `lead_time` is the variable most clearly distributed around **specific values**, with distinct peaks.
* The most common lead times appear to be 2, 4, and 5 units of time (days or weeks, depending on the unit). The peaks at 4 and 5 are the highest.
* This indicates that supply logistics are **clustered around fixed timeframes**, which may reflect transportation schedules, production cycles, or SLAs (Service Level Agreements) with suppliers.

### 6. `doi_inventory_turnover`
* The distribution of `doi_inventory_turnover` is **scattered**, with a long tail.
* Most products have a DOI in the range of 0 to 2000, with peaks around 500, 1000, and 1500.
* A lower DOI is generally better, indicating faster inventory turnover. The wide dispersion suggests **significant variation in turnover speed** across products.

### 7. `delivery_lag`
* This distribution shows delivery delays in whole numbers (likely days).
* There is **uniformity in shorter delays** (from 2 to 10 days). All integer values in this range have very similar frequencies (around 200 occurrences).
* This may suggest that delays are **rounded** or that there is a delivery buffer policy, where small delays are common and evenly distributed. The absence of extreme peaks is a good sign that major delays are not prevalent.

### 8. `maximum_days_on_sale`
* The distribution is **heavily left-skewed**, with the vast majority of products having a very low maximum number of days on sale (in the range of 0 to 50 days).
* This may indicate that most products **turn over quickly**, or that the data analyzed mostly covers products with a recent and therefore short sales history.

### 9. `inventory_turnover_rate`
* This distribution is also **heavily left-skewed**, with the highest frequency concentrated in **low turnover rates** (likely 0 to 0.5).
* This is expected, as most products will not have a very high turnover.
* There is a tail extending to higher values (above 2.0), indicating that **a few high-turnover products** contribute significantly to revenue.

---

### General Conclusions

The data suggests a supply chain with the following characteristics:

1. **Focus on Slow/Medium Turnover Items:** Most products have low maximum stock, minimum stock, and turnover rates, indicating that the product portfolio is primarily composed of slow to moderately moving items.
2. **Rigid Lead Time Logistics:** The concentration of lead times around a few values (2, 4, and 5) suggests well-defined and possibly rigid procurement and logistics processes.
3. **Conservative Inventory Control:** The concentration of `max_stock` and `min_stock` at low values indicates an effort to minimize excess inventory, consistent with the high volume of products with low `shelf_life_days` and low `inventory_turnover_rate`.
4. **Common Delivery Delays:** The uniform distribution of `delivery_lag` suggests that minor delays (2 to 10 days) are a stable and predictable feature of operations.

---

